# IMPORTS

In [ ]:
import numpy as np
from datetime import datetime
from pathlib import Path
from IPython.display import clear_output
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.models import load_model
from visualkeras import layered_view
from keras.datasets import mnist, fashion_mnist, cifar10
from keras.utils.vis_utils import plot_model
from sklearn.utils import shuffle

from gand.config import MLConfig
from gand.data import data
from gand.models import models, architecture
from gand.visualisation import visualise
from gand.preprocessing import utils

import pandas as pd
from tabulate import tabulate
from tqdm.keras import TqdmCallback

from sklearn.metrics import classification_report


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
dataset = mnist
EPOCHS = 100
BATCH_SIZE = MLConfig.BATCH_SIZE

loss = "categorical_crossentropy"
opt = tf.keras.optimizers.legacy.SGD(learning_rate=0.0001, momentum=0.9)
metrics = ["accuracy"]

g_model_mnist = load_model(Path.cwd() / 'notebooks/models/cgan/mnist/gen_model_e-200.h5')
g_model_fashion_mnist = load_model(Path.cwd() / 'notebooks/models/cgan/fashion_mnist/gen_model_e-536.h5')
g_model_cifar10 = load_model(Path.cwd() / 'notebooks/models/cgan/cifar10/gen_model_e-553.h5')
g_model = g_model_mnist

train_with_gan = False
imbalance_data = False
train_type = 0

In [ ]:
name = 'normal'
if train_with_gan:
    name = 'gans'
elif imbalance_data:
    name = 'imbalanced'

MLConfig.TYPE_NAMES = [f'{name}']
print(MLConfig.TYPE_NAMES)

# Data Processing

### GAN Data

In [ ]:
X_train_gan, y_train_gan = None, None
if train_with_gan:
    imbalance_data = True
    # GAN DATA
    n = 4000
    X_train_gan, y_train_gan = data.generate_fake_data(n=n, g_model=g_model, seed=10, verbose=1)
    X_train_gan, y_train_gan = utils.preprocess_data(X_train_gan, y_train_gan, val_255=False, exp_dims=False)
    
    print(X_train_gan.min(), X_train_gan.max(), X_train_gan.shape, y_train_gan.shape)

### Normal Data

In [ ]:
((X_train_real, y_train_real), (X_test_real, y_test_real)), dataset_name = data.load_dataset(dataset, return_name=True)
print(X_train_real.shape, y_train_real.shape, X_test_real.shape, y_test_real.shape, dataset_name)

exp_dims = False
if len(X_train_real.shape) == 3:
    exp_dims = True


print(X_train_real.min(), X_train_real.max(), X_test_real.min(), X_test_real.max())

X_train_real, y_train_real = utils.preprocess_data(X_train_real, y_train_real, exp_dims=exp_dims)
X_test_real, y_test_real = utils.preprocess_data(X_test_real, y_test_real, exp_dims=exp_dims)

print(X_train_real.min(), X_train_real.max(), X_test_real.min(), X_test_real.max())
print(X_train_real.shape, y_train_real.shape, X_test_real.shape, y_test_real.shape, dataset_name)

if imbalance_data:
    # REMOVING 75%
    keep_per_class = 1000
    X_train_removed, y_train_removed = [], []
    for i in range(y_train_real.shape[-1]):
        class_idx = np.where(np.argmax(y_train_real, axis=-1) == i)[0]
        selected_idx = np.random.choice(class_idx, keep_per_class)
        X_train_removed.append(X_train_real[selected_idx])
        y_train_removed.append(y_train_real[selected_idx])
    
    X_train_real, y_train_real = np.concatenate(X_train_removed, axis=0), np.concatenate(y_train_removed, axis=0)
print(X_train_real.shape, y_train_real.shape, X_test_real.shape, y_test_real.shape, dataset_name)

### Appending GAN Data

In [ ]:
if train_with_gan:
    X_train_real = np.concatenate((X_train_real, X_train_gan), axis=0)
    y_train_real = np.concatenate((y_train_real, y_train_gan), axis=0)
    print(X_train_real.shape, y_train_real.shape, X_test_real.shape, y_test_real.shape, dataset_name)

### Shuffle It

In [ ]:
X_train_real, y_train_real = shuffle(X_train_real, y_train_real)
X_test_real, y_test_real = shuffle(X_test_real, y_test_real)
print(X_train_real.shape, y_train_real.shape, X_test_real.shape, y_test_real.shape, dataset_name)

### Make it Faster

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((X_train_real, y_train_real))
test_data = tf.data.Dataset.from_tensor_slices((X_test_real, y_test_real))

train_data = train_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_data, test_data, dataset_name

In [ ]:
MLConfig.TYPE_NAMES[train_type], X_train_real.shape[1:]

In [ ]:
model = architecture.deep_model(img_shape=X_train_real.shape[1:])
model.compile(loss=loss, optimizer=opt, metrics=metrics)
model.name, EPOCHS, BATCH_SIZE

# Fitting the Model

In [ ]:
EPOCHS = 100
history = model.fit(X_train_real, y_train_real, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_data=(X_test_real, y_test_real), verbose=0,
                    callbacks=[TqdmCallback(verbose=1)], shuffle=True)

## Saving Evaluations

In [ ]:
eval_data = ((X_train_real, y_train_real), (X_test_real, y_test_real))
models.save_metrics(dataset_name=dataset_name, train_type=train_type,
                   epochs=EPOCHS, model=model, eval_data=eval_data, 
                   history=history)

## Saving History

In [ ]:
history_path = Path.cwd() / f'reports/history/{dataset_name}/{MLConfig.TYPE_NAMES[train_type]}/E_{EPOCHS:03d}/'
history_path.mkdir(parents=True, exist_ok=True)
pd.DataFrame.from_dict(history.history).to_csv(history_path / f'{model.name}.csv',index=False)

# Visualisation

In [ ]:
visualise.metric_plot(show_fig=True, history=history, dataset_name=dataset_name, 
                      savefig=False, epochs=EPOCHS)

# Evaluation

In [ ]:
# model = load_model(Path.cwd() / Path(f'reports/models/{dataset_name}/{MLConfig.TYPE_NAMES[train_type]}/E_{EPOCHS:03d}/{model.name}.h5'))
# model.name

In [ ]:
_, acc = model.evaluate(X_train_real, y_train_real)
_, acc = model.evaluate(X_test_real, y_test_real)

y_pred = np.argmax(model.predict(X_train_real), axis=-1)
report = classification_report(np.argmax(y_train_real, axis=1), y_pred)
print(report)

y_pred = np.argmax(model.predict(X_test_real), axis=-1)
report = classification_report(np.argmax(y_test_real, axis=1), y_pred)
print(report)